In [1]:
!pip install keras

In [2]:
!pip install tensorflow

In [1]:
import math, json, os, sys

import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
import tensorflow

In [2]:
import pandas as pd
from tensorflow.keras.applications.regnet import RegNetX002

In [3]:

DATA_DIR = 'C:\\Users\\choig\\Downloads\\fine_tune_data'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VALID_DIR = os.path.join(DATA_DIR, 'validation')
SIZE = (224, 224)
BATCH_SIZE = 32

In [4]:
VALID_DIR

'C:\\Users\\choig\\Downloads\\fine_tune_data\\validation'

In [5]:
TRAIN_DIR

'C:\\Users\\choig\\Downloads\\fine_tune_data\\train'

In [6]:

DATA_DIR = 'fine_tune_data'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
VALID_DIR = os.path.join(DATA_DIR, 'validation')
SIZE = (224, 224)
BATCH_SIZE = 32


if __name__ == "__main__":
    num_train_samples = sum([len(files) for r, d, files in os.walk(TRAIN_DIR)])
    num_valid_samples = sum([len(files) for r, d, files in os.walk(VALID_DIR)])

    num_train_steps = math.floor(num_train_samples/BATCH_SIZE)
    num_valid_steps = math.floor(num_valid_samples/BATCH_SIZE)

    gen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=40, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2)
    val_gen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=40, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2)

    batches = gen.flow_from_directory(TRAIN_DIR, target_size=SIZE, class_mode='binary', shuffle=True, batch_size=BATCH_SIZE)
    val_batches = val_gen.flow_from_directory(VALID_DIR, target_size=SIZE, class_mode='binary', shuffle=True, batch_size=BATCH_SIZE)

    model = tensorflow.keras.applications.regnet.RegNetX002()

    classes = list(iter(batches.class_indices))
    model.layers.pop()
    for layer in model.layers:
        layer.trainable=False
    last = model.layers[-1].output
    #x = Dense(len(classes), activation="softmax")(last)
    x = Dense(1, activation="sigmoid")(last)
    finetuned_model = Model(model.input, x)
    #finetuned_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    finetuned_model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    for c in batches.class_indices:
        classes[batches.class_indices[c]] = c
    finetuned_model.classes = classes

    early_stopping = EarlyStopping(patience=10)
    checkpointer = ModelCheckpoint('./regnet_best.h5', verbose=1, save_best_only=True)

    finetuned_model.fit_generator(batches, steps_per_epoch=num_train_steps, epochs=300, callbacks=[early_stopping, checkpointer], validation_data=val_batches, validation_steps=num_valid_steps)
    finetuned_model.save('regnet_final.h5')

Found 9600 images belonging to 2 classes.
Found 1201 images belonging to 2 classes.
11278832/11278832 [==============================] - 1s 0us/step


C:\Users\choig\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\choig\AppData\Local\Temp\ipykernel_11652\3093217643.py:40: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  finetuned_model.fit_generator(batches, steps_per_epoch=num_train_steps, epochs=300, callbacks=[early_stopping, checkpointer], validation_data=val_batches, validation_steps=num_valid_steps)


Epoch 1/300
300/300 [==============================] - ETA: 0s - loss: 1.0252 - accuracy: 0.5846
Epoch 1: val_loss improved from inf to 0.81718, saving model to .\regnet_best.h5
300/300 [==============================] - 442s 1s/step - loss: 1.0252 - accuracy: 0.5846 - val_loss: 0.8172 - val_accuracy: 0.6588
Epoch 2/300
300/300 [==============================] - ETA: 0s - loss: 0.6808 - accuracy: 0.7036
Epoch 2: val_loss improved from 0.81718 to 0.59486, saving model to .\regnet_best.h5
300/300 [==============================] - 432s 1s/step - loss: 0.6808 - accuracy: 0.7036 - val_loss: 0.5949 - val_accuracy: 0.7500
Epoch 3/300
300/300 [==============================] - ETA: 0s - loss: 0.5606 - accuracy: 0.7592
Epoch 3: val_loss improved from 0.59486 to 0.55767, saving model to .\regnet_best.h5
300/300 [==============================] - 432s 1s/step - loss: 0.5606 - accuracy: 0.7592 - val_loss: 0.5577 - val_accuracy: 0.7644
Epoch 4/300
300/300 [==============================] - ETA: 0s

300/300 [==============================] - 418s 1s/step - loss: 0.3347 - accuracy: 0.8562 - val_loss: 0.3273 - val_accuracy: 0.8615
Epoch 29/300
300/300 [==============================] - ETA: 0s - loss: 0.3359 - accuracy: 0.8570
Epoch 29: val_loss did not improve from 0.32727
300/300 [==============================] - 424s 1s/step - loss: 0.3359 - accuracy: 0.8570 - val_loss: 0.3343 - val_accuracy: 0.8615
Epoch 30/300
300/300 [==============================] - ETA: 0s - loss: 0.3330 - accuracy: 0.8581
Epoch 30: val_loss did not improve from 0.32727
300/300 [==============================] - 429s 1s/step - loss: 0.3330 - accuracy: 0.8581 - val_loss: 0.3278 - val_accuracy: 0.8564
Epoch 31/300
300/300 [==============================] - ETA: 0s - loss: 0.3288 - accuracy: 0.8621
Epoch 31: val_loss did not improve from 0.32727
300/300 [==============================] - 437s 1s/step - loss: 0.3288 - accuracy: 0.8621 - val_loss: 0.3592 - val_accuracy: 0.8471
Epoch 32/300
300/300 [============

300/300 [==============================] - ETA: 0s - loss: 0.3156 - accuracy: 0.8703
Epoch 57: val_loss did not improve from 0.30195
300/300 [==============================] - 429s 1s/step - loss: 0.3156 - accuracy: 0.8703 - val_loss: 0.3239 - val_accuracy: 0.8598
Epoch 58/300
300/300 [==============================] - ETA: 0s - loss: 0.3148 - accuracy: 0.8660
Epoch 58: val_loss did not improve from 0.30195
300/300 [==============================] - 437s 1s/step - loss: 0.3148 - accuracy: 0.8660 - val_loss: 0.3196 - val_accuracy: 0.8699
Epoch 59/300
300/300 [==============================] - ETA: 0s - loss: 0.3057 - accuracy: 0.8735
Epoch 59: val_loss did not improve from 0.30195
300/300 [==============================] - 430s 1s/step - loss: 0.3057 - accuracy: 0.8735 - val_loss: 0.3026 - val_accuracy: 0.8682
Epoch 60/300
300/300 [==============================] - ETA: 0s - loss: 0.3030 - accuracy: 0.8689
Epoch 60: val_loss did not improve from 0.30195
300/300 [========================

In [8]:
from keras.models import load_model

In [9]:
finetuned_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 regnetx002_prestem_rescaling (  (None, 224, 224, 3)  0          ['input_1[0][0]']                
 Rescaling)                                                                                       
                                                                                                  
 regnetx002_stem_conv (Conv2D)  (None, 112, 112, 32  864         ['regnetx002_prestem_rescaling[0]
                                )                                [0]']                        

 regnetx002_Stage_1_XBlock_0_co  (None, 28, 28, 56)  224         ['regnetx002_Stage_1_XBlock_0_con
 nv_1x1_2_bn (BatchNormalizatio                                  v_1x1_2[0][0]']                  
 n)                                                                                               
                                                                                                  
 regnetx002_Stage_1_XBlock_0_sk  (None, 28, 28, 56)  224         ['regnetx002_Stage_1_XBlock_0_ski
 ip_bn (BatchNormalization)                                      p_1x1[0][0]']                    
                                                                                                  
 tf.__operators__.add_1 (TFOpLa  (None, 28, 28, 56)  0           ['regnetx002_Stage_1_XBlock_0_con
 mbda)                                                           v_1x1_2_bn[0][0]',               
                                                                  'regnetx002_Stage_1_XBlock_0_ski
          

                                                                  'regnetx002_Stage_2_XBlock_0_exi
                                                                 t_relu[0][0]']                   
                                                                                                  
 regnetx002_Stage_2_XBlock_1_ex  (None, 14, 14, 152)  0          ['tf.__operators__.add_3[0][0]'] 
 it_relu (ReLU)                                                                                   
                                                                                                  
 regnetx002_Stage_2_XBlock_2_co  (None, 14, 14, 152)  23104      ['regnetx002_Stage_2_XBlock_1_exi
 nv_1x1_1 (Conv2D)                                               t_relu[0][0]']                   
                                                                                                  
 regnetx002_Stage_2_XBlock_2_co  (None, 14, 14, 152)  608        ['regnetx002_Stage_2_XBlock_2_con
 nv_1x1_1_

                                                                                                  
 regnetx002_Stage_3_XBlock_0_co  (None, 7, 7, 368)   26496       ['regnetx002_Stage_3_XBlock_0_con
 nv_3x3 (Conv2D)                                                 v_1x1_1_relu[0][0]']             
                                                                                                  
 regnetx002_Stage_3_XBlock_0_co  (None, 7, 7, 368)   1472        ['regnetx002_Stage_3_XBlock_0_con
 nv_3x3_bn (BatchNormalization)                                  v_3x3[0][0]']                    
                                                                                                  
 regnetx002_Stage_3_XBlock_0_co  (None, 7, 7, 368)   0           ['regnetx002_Stage_3_XBlock_0_con
 nv_3x3_relu (ReLU)                                              v_3x3_bn[0][0]']                 
                                                                                                  
 regnetx00

                                                                                                  
 regnetx002_Stage_3_XBlock_2_co  (None, 7, 7, 368)   135424      ['regnetx002_Stage_3_XBlock_2_con
 nv_1x1_2 (Conv2D)                                               v_3x3_relu[0][0]']               
                                                                                                  
 regnetx002_Stage_3_XBlock_2_co  (None, 7, 7, 368)   1472        ['regnetx002_Stage_3_XBlock_2_con
 nv_1x1_2_bn (BatchNormalizatio                                  v_1x1_2[0][0]']                  
 n)                                                                                               
                                                                                                  
 tf.__operators__.add_8 (TFOpLa  (None, 7, 7, 368)   0           ['regnetx002_Stage_3_XBlock_2_con
 mbda)                                                           v_1x1_2_bn[0][0]',               
          

                                                                                                  
 regnetx002_Stage_3_XBlock_5_co  (None, 7, 7, 368)   135424      ['regnetx002_Stage_3_XBlock_4_exi
 nv_1x1_1 (Conv2D)                                               t_relu[0][0]']                   
                                                                                                  
 regnetx002_Stage_3_XBlock_5_co  (None, 7, 7, 368)   1472        ['regnetx002_Stage_3_XBlock_5_con
 nv_1x1_1_bn (BatchNormalizatio                                  v_1x1_1[0][0]']                  
 n)                                                                                               
                                                                                                  
 regnetx002_Stage_3_XBlock_5_co  (None, 7, 7, 368)   0           ['regnetx002_Stage_3_XBlock_5_con
 nv_1x1_1_relu (ReLU)                                            v_1x1_1_bn[0][0]']               
          